In [158]:
class Note:
    
    def __init__(self, channel, pitch, velocity, timestamp, duration):
        self.channel = channel
        self.pitch = pitch
        self.velocity = velocity
        
        # in MIDI 'ticks'
        self.timestamp = timestamp
        self.duration = duration
        
    def __eq__(self, other):
        return self.channel == other.channel and self.pitch == other.pitch and self.velocity == other.velocity and \
            self.duration == other.duration
        
    def __repr__(self):
        return f"[Note {self.channel} {self.pitch} {self.velocity} {self.duration}]"
        
    def __hash__(self):
        return hash(str(self))

In [175]:
from random import randint
from midiutil import MIDIFile

class MarkovGenerator:
    
    def __init__(self, order, note_lst):
        self.order = order
        self.note_lst = note_lst
        self.adj = {}
        
        self.build_graph()
        
    def build_graph(self):
        
        for i in range(len(note_lst)):
            
            for j in range(i+1, i+1+self.order):
                
                note_slice = tuple(note_lst[i:j])
                
                if j+1 >= len(note_lst):
                    # out of bounds
                    break
                    
                nxt_note = note_lst[j+1]
                
                if note_slice not in self.adj:
                    self.adj[note_slice] = []
                    
                self.adj[note_slice].append(nxt_note)
                
    def generate_notes(self, song_length):
        
        output_notes = self.note_lst[:self.order]
        
        for _ in range(song_length):
            
            start_idx = max(0, len(output_notes) - self.order)
            end_idx = min(len(output_notes), start_idx + self.order)
            
            while start_idx < end_idx:
                
                note_slice = tuple(output_notes[start_idx: end_idx])
                
                # print("trying", note_slice)
            
                if note_slice in self.adj:
                    
                    # found a match in our adjacency list
                    
                    print("using", start_idx, end_idx, note_slice, output_notes[-3:])
                    
                    options = self.adj[note_slice]
                    output_notes.append(options[randint(0, len(options) - 1)])
                    
                    break
                
                start_idx += 1
        
        return output_notes
    
    def generate_midi(self, song_length):
    
        notes = self.generate_notes(song_length)

        track = 0
        time = 0
        tempo = 60 # In BPM

        MyMIDI = MIDIFile(1, eventtime_is_ticks=True)
        MyMIDI.addTempo(track,time,tempo)

        for note in notes:

            MyMIDI.addNote(track, note.channel, note.pitch, note.timestamp, note.duration, note.velocity)

        return MyMIDI

In [176]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile('the_real_folk_blues.mid')

note_set = set()
note_lst = []

open_notes = {}

counter = 0
cur_tick = 0

for msg in mid.tracks[0]:
    
    cur_tick += msg.time
    
    if msg.type == 'note_on' and msg.velocity != 0:
       
        note = Note(msg.channel, msg.note, msg.velocity, cur_tick, 0)
        open_notes[(msg.channel, msg.note)] = note
        counter += 1
        
    elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
        
        note_id = (msg.channel, msg.note)
        
        if note_id in open_notes:
            open_notes[note_id].duration = cur_tick - open_notes[note_id].timestamp
            note_set.add(open_notes[note_id])
            note_lst.append(open_notes[note_id])
            
            open_notes.pop(note_id)
        
print(f"Total notes: {counter}")
print(f"Unique notes: {len(note_set)}")

note_lst.sort(key=lambda x: x.timestamp)

generator = MarkovGenerator(10, note_lst)
output_mid = generator.generate_midi(2000)

# output_mid = generate_midi(note_lst)

with open("folk_blue_generated.mid", "wb") as output_file:
    output_mid.writeFile(output_file)
    
print('finished')

Total notes: 2100
Unique notes: 322
using 0 10 ([Note 0 68 127 220], [Note 0 64 127 220], [Note 0 61 127 220], [Note 0 57 127 220], [Note 1 54 127 220], [Note 0 68 127 100], [Note 0 64 127 100], [Note 0 61 127 100], [Note 0 57 127 100], [Note 1 54 127 100]) [[Note 0 61 127 100], [Note 0 57 127 100], [Note 1 54 127 100]]
using 10 11 ([Note 0 64 127 3100],) [[Note 0 57 127 100], [Note 1 54 127 100], [Note 0 64 127 3100]]
using 11 12 ([Note 0 57 127 3100],) [[Note 1 54 127 100], [Note 0 64 127 3100], [Note 0 57 127 3100]]
using 12 13 ([Note 1 42 100 240],) [[Note 0 64 127 3100], [Note 0 57 127 3100], [Note 1 42 100 240]]
using 13 14 ([Note 0 61 127 450],) [[Note 0 57 127 3100], [Note 1 42 100 240], [Note 0 61 127 450]]
using 14 15 ([Note 1 49 100 100],) [[Note 1 42 100 240], [Note 0 61 127 450], [Note 1 49 100 100]]
using 15 16 ([Note 1 54 100 100],) [[Note 0 61 127 450], [Note 1 49 100 100], [Note 1 54 100 100]]
using 16 17 ([Note 3 66 100 600],) [[Note 1 49 100 100], [Note 1 54 100 100]

using 263 264 ([Note 3 66 100 120],) [[Note 0 81 127 240], [Note 1 54 100 340], [Note 3 66 100 120]]
using 264 265 ([Note 0 69 127 240],) [[Note 1 54 100 340], [Note 3 66 100 120], [Note 0 69 127 240]]
using 265 266 ([Note 1 54 100 100],) [[Note 3 66 100 120], [Note 0 69 127 240], [Note 1 54 100 100]]
using 266 267 ([Note 2 88 127 100],) [[Note 0 69 127 240], [Note 1 54 100 100], [Note 2 88 127 100]]
using 267 268 ([Note 1 49 100 340],) [[Note 1 54 100 100], [Note 2 88 127 100], [Note 1 49 100 340]]
using 268 269 ([Note 3 66 100 120],) [[Note 2 88 127 100], [Note 1 49 100 340], [Note 3 66 100 120]]
using 269 270 ([Note 3 64 100 120],) [[Note 1 49 100 340], [Note 3 66 100 120], [Note 3 64 100 120]]
using 270 271 ([Note 0 66 127 120],) [[Note 3 66 100 120], [Note 3 64 100 120], [Note 0 66 127 120]]
using 271 272 ([Note 1 47 100 100],) [[Note 3 64 100 120], [Note 0 66 127 120], [Note 1 47 100 100]]
using 272 273 ([Note 0 71 127 120],) [[Note 0 66 127 120], [Note 1 47 100 100], [Note 0 71 

using 463 464 ([Note 3 66 100 120],) [[Note 1 37 100 100], [Note 1 40 100 220], [Note 3 66 100 120]]
using 464 465 ([Note 3 64 100 120],) [[Note 1 40 100 220], [Note 3 66 100 120], [Note 3 64 100 120]]
using 465 466 ([Note 0 63 127 1920],) [[Note 3 66 100 120], [Note 3 64 100 120], [Note 0 63 127 1920]]
using 466 467 ([Note 3 61 100 120],) [[Note 3 64 100 120], [Note 0 63 127 1920], [Note 3 61 100 120]]
using 467 468 ([Note 0 68 127 120],) [[Note 0 63 127 1920], [Note 3 61 100 120], [Note 0 68 127 120]]
using 468 469 ([Note 1 47 100 100],) [[Note 3 61 100 120], [Note 0 68 127 120], [Note 1 47 100 100]]
using 469 470 ([Note 3 59 100 120],) [[Note 0 68 127 120], [Note 1 47 100 100], [Note 3 59 100 120]]
using 470 471 ([Note 3 61 100 240],) [[Note 1 47 100 100], [Note 3 59 100 120], [Note 3 61 100 240]]
using 471 472 ([Note 0 66 127 120],) [[Note 3 59 100 120], [Note 3 61 100 240], [Note 0 66 127 120]]
using 472 473 ([Note 0 69 127 120],) [[Note 3 61 100 240], [Note 0 66 127 120], [Note 0

using 663 664 ([Note 0 64 127 1900],) [[Note 0 69 127 240], [Note 1 44 100 100], [Note 0 64 127 1900]]
using 664 665 ([Note 0 59 127 1900],) [[Note 1 44 100 100], [Note 0 64 127 1900], [Note 0 59 127 1900]]
using 665 666 ([Note 1 52 100 100],) [[Note 0 64 127 1900], [Note 0 59 127 1900], [Note 1 52 100 100]]
using 666 667 ([Note 0 61 127 340],) [[Note 0 59 127 1900], [Note 1 52 100 100], [Note 0 61 127 340]]
using 667 668 ([Note 1 42 100 340],) [[Note 1 52 100 100], [Note 0 61 127 340], [Note 1 42 100 340]]
using 668 669 ([Note 3 71 100 240],) [[Note 0 61 127 340], [Note 1 42 100 340], [Note 3 71 100 240]]
using 669 670 ([Note 3 73 100 240],) [[Note 1 42 100 340], [Note 3 71 100 240], [Note 3 73 100 240]]
using 670 671 ([Note 0 76 127 240],) [[Note 3 71 100 240], [Note 3 73 100 240], [Note 0 76 127 240]]
using 671 672 ([Note 1 50 100 100],) [[Note 3 73 100 240], [Note 0 76 127 240], [Note 1 50 100 100]]
using 672 673 ([Note 0 71 127 360],) [[Note 0 76 127 240], [Note 1 50 100 100], [No

using 863 864 ([Note 0 61 127 450],) [[Note 1 52 100 100], [Note 1 42 100 220], [Note 0 61 127 450]]
using 864 865 ([Note 1 49 100 100],) [[Note 1 42 100 220], [Note 0 61 127 450], [Note 1 49 100 100]]
using 864 866 ([Note 1 49 100 100], [Note 1 52 100 100]) [[Note 0 61 127 450], [Note 1 49 100 100], [Note 1 52 100 100]]
using 866 867 ([Note 1 49 100 100],) [[Note 1 49 100 100], [Note 1 52 100 100], [Note 1 49 100 100]]
using 867 868 ([Note 3 59 100 120],) [[Note 1 52 100 100], [Note 1 49 100 100], [Note 3 59 100 120]]
using 867 869 ([Note 3 59 100 120], [Note 0 66 127 120]) [[Note 1 49 100 100], [Note 3 59 100 120], [Note 0 66 127 120]]
using 868 870 ([Note 0 66 127 120], [Note 1 54 100 220]) [[Note 3 59 100 120], [Note 0 66 127 120], [Note 1 54 100 220]]
using 869 871 ([Note 1 54 100 220], [Note 1 42 100 100]) [[Note 0 66 127 120], [Note 1 54 100 220], [Note 1 42 100 100]]
using 870 872 ([Note 1 42 100 100], [Note 1 42 100 100]) [[Note 1 54 100 220], [Note 1 42 100 100], [Note 1 42 1

...

using 1263 1264 ([Note 0 73 127 340],) [[Note 1 57 100 220], [Note 3 66 100 240], [Note 0 73 127 340]]
using 1264 1265 ([Note 1 49 100 100],) [[Note 3 66 100 240], [Note 0 73 127 340], [Note 1 49 100 100]]
using 1265 1266 ([Note 1 54 100 100],) [[Note 0 73 127 340], [Note 1 49 100 100], [Note 1 54 100 100]]
using 1266 1267 ([Note 3 66 100 600],) [[Note 1 49 100 100], [Note 1 54 100 100], [Note 3 66 100 600]]
using 1266 1268 ([Note 3 66 100 600], [Note 1 47 100 220]) [[Note 1 54 100 100], [Note 3 66 100 600], [Note 1 47 100 220]]
using 1267 1269 ([Note 1 47 100 220], [Note 0 69 127 220]) [[Note 3 66 100 600], [Note 1 47 100 220], [Note 0 69 127 220]]
using 1268 1270 ([Note 0 69 127 220], [Note 3 64 100 220]) [[Note 1 47 100 220], [Note 0 69 127 220], [Note 3 64 100 220]]
using 1270 1271 ([Note 1 42 100 100],) [[Note 0 69 127 220], [Note 3 64 100 220], [Note 1 42 100 100]]
using 1270 1272 ([Note 1 42 100 100], [Note 0 61 127 450]) [[Note 3 64 100 220], [Note 1 42 100 100], [Note 0 61 127

using 1483 1484 ([Note 1 52 100 100],) [[Note 1 42 100 220], [Note 0 61 127 450], [Note 1 52 100 100]]
using 1484 1485 ([Note 3 68 100 570],) [[Note 0 61 127 450], [Note 1 52 100 100], [Note 3 68 100 570]]
using 1484 1486 ([Note 3 68 100 570], [Note 1 45 100 220]) [[Note 1 52 100 100], [Note 3 68 100 570], [Note 1 45 100 220]]
using 1485 1487 ([Note 1 45 100 220], [Note 1 57 100 220]) [[Note 3 68 100 570], [Note 1 45 100 220], [Note 1 57 100 220]]
using 1487 1488 ([Note 1 37 100 240],) [[Note 1 45 100 220], [Note 1 57 100 220], [Note 1 37 100 240]]
using 1488 1489 ([Note 0 57 127 450],) [[Note 1 57 100 220], [Note 1 37 100 240], [Note 0 57 127 450]]
using 1489 1490 ([Note 1 54 100 220],) [[Note 1 37 100 240], [Note 0 57 127 450], [Note 1 54 100 220]]
using 1489 1491 ([Note 1 54 100 220], [Note 0 69 127 240]) [[Note 0 57 127 450], [Note 1 54 100 220], [Note 0 69 127 240]]
using 1491 1492 ([Note 1 42 100 220],) [[Note 1 54 100 220], [Note 0 69 127 240], [Note 1 42 100 220]]
using 1492 14

using 1662 1663 ([Note 1 57 100 340],) [[Note 0 64 127 1900], [Note 0 59 127 1900], [Note 1 57 100 340]]
using 1663 1664 ([Note 1 45 100 100],) [[Note 0 59 127 1900], [Note 1 57 100 340], [Note 1 45 100 100]]
using 1664 1665 ([Note 0 62 127 700],) [[Note 1 57 100 340], [Note 1 45 100 100], [Note 0 62 127 700]]
using 1665 1666 ([Note 0 57 127 700],) [[Note 1 45 100 100], [Note 0 62 127 700], [Note 0 57 127 700]]
using 1666 1667 ([Note 1 54 100 340],) [[Note 0 62 127 700], [Note 0 57 127 700], [Note 1 54 100 340]]
using 1667 1668 ([Note 3 66 100 120],) [[Note 0 57 127 700], [Note 1 54 100 340], [Note 3 66 100 120]]
using 1668 1669 ([Note 0 66 127 600],) [[Note 1 54 100 340], [Note 3 66 100 120], [Note 0 66 127 600]]
using 1669 1670 ([Note 1 42 100 220],) [[Note 3 66 100 120], [Note 0 66 127 600], [Note 1 42 100 220]]
using 1670 1671 ([Note 0 69 127 100],) [[Note 0 66 127 600], [Note 1 42 100 220], [Note 0 69 127 100]]
using 1671 1672 ([Note 0 78 127 600],) [[Note 1 42 100 220], [Note 0 6

using 1862 1863 ([Note 0 69 127 960],) [[Note 3 71 100 240], [Note 1 51 100 220], [Note 0 69 127 960]]
using 1863 1864 ([Note 1 45 100 220],) [[Note 1 51 100 220], [Note 0 69 127 960], [Note 1 45 100 220]]
using 1863 1865 ([Note 1 45 100 220], [Note 1 45 100 220]) [[Note 0 69 127 960], [Note 1 45 100 220], [Note 1 45 100 220]]
using 1865 1866 ([Note 0 73 127 120],) [[Note 1 45 100 220], [Note 1 45 100 220], [Note 0 73 127 120]]
using 1866 1867 ([Note 1 54 100 100],) [[Note 1 45 100 220], [Note 0 73 127 120], [Note 1 54 100 100]]
using 1866 1868 ([Note 1 54 100 100], [Note 1 52 100 100]) [[Note 0 73 127 120], [Note 1 54 100 100], [Note 1 52 100 100]]
using 1868 1869 ([Note 0 61 127 340],) [[Note 1 54 100 100], [Note 1 52 100 100], [Note 0 61 127 340]]
using 1869 1870 ([Note 1 49 100 100],) [[Note 1 52 100 100], [Note 0 61 127 340], [Note 1 49 100 100]]
using 1870 1871 ([Note 3 59 100 120],) [[Note 0 61 127 340], [Note 1 49 100 100], [Note 3 59 100 120]]
using 1870 1872 ([Note 3 59 100 1

finished


In [172]:
from midiutil import MIDIFile

def generate_midi(notes):
    
    if not notes:
        return

    track = 0
    time = 0
    tempo = 60 # In BPM
    
    MyMIDI = MIDIFile(1, eventtime_is_ticks=True)
    MyMIDI.addTempo(track,time,tempo)
    
    for note in notes:
        
        MyMIDI.addNote(track, note.channel, note.pitch, note.timestamp, note.duration, note.velocity)
   
    return MyMIDI